In [4]:
from operator import itemgetter

def parse_result_file_sam(filepath):
    reads_info = dict()
    with open(filepath) as f:
        f.readline()
        f.readline()
        line = f.readline()
        while line:
            tokens = line.split("\t")
            reads_info_entry = dict()
            reads_info_entry["flags"] = int(tokens[1]) & 0x10
            reads_info_entry["fasta_seq"] = tokens[2]
            reads_info_entry["start_ref"] = int(tokens[3])
            reads_info_entry["map_qual"] = tokens[4]
            reads_info_entry["cigar"] = tokens[5]
            reads_info_entry["length"] = int(len(tokens[9]))
            if tokens[0] in reads_info:
                reads_info[tokens[0]].append(reads_info_entry)
            else:
                reads_info[tokens[0]] = [reads_info_entry]
            line = f.readline()

    return reads_info


def parse_result_file_custom(filepath):
    reads_info = dict()
    with open(filepath) as f:
        line = f.readline()
        while line:
            tokens = line.split("\t")
            reads_info_entry = dict()
            reads_info_entry["flags"] = int(tokens[1])
            reads_info_entry["fasta_seq"] = tokens[2]
            reads_info_entry["start_ref"] = int(tokens[3])
            reads_info_entry["map_qual"] = tokens[4]
            reads_info_entry["cigar"] = tokens[5]
            reads_info_entry["length"] = int(len(tokens[6]))
            if tokens[0] in reads_info:
                reads_info[tokens[0]].append(reads_info_entry)
            else:
                reads_info[tokens[0]] = [reads_info_entry]
            line = f.readline()
            
    return reads_info

        
def compare_results(fasta_filepath, bwa_filepath, custom_filepath):
    with open(fasta_filepath) as f:
        f.readline()
        fasta_line = f.readline()
        
    fasta_seq_len = len(fasta_line)
    bwa_reads_info = parse_result_file_sam(bwa_filepath)
    custom_reads_info = parse_result_file_custom(custom_filepath)
    
    n_bwa = len(bwa_reads_info)
    len_diff_cnt = 0
    corr_map_cnt = 0
    custom_cnt = 0
    
    for key in bwa_reads_info:
        if key in custom_reads_info:
            custom_cnt += 1
            if len(bwa_reads_info[key]) != len(custom_reads_info[key]):
                len_diff_cnt += 1
                """print("len diff:")
                print("bwa: " + key + " => " + str(len(bwa_reads_info[key])))
                print("cst: " + key + " => " + str(len(custom_reads_info[key])))"""
                for i in range(len(bwa_reads_info[key])):
                    for j in range(len(custom_reads_info[key])):
                        bwa_entry = bwa_reads_info[key][i]
                        custom_entry = custom_reads_info[key][j]
                        
                        bwa_start = max(0, bwa_entry["start_ref"] - bwa_entry["length"])
                        bwa_end = min(fasta_seq_len, bwa_entry["start_ref"] + bwa_entry["length"])
                        if bwa_start <= custom_entry["start_ref"] and custom_entry["start_ref"] <= bwa_end:
                            corr_map_cnt += 1
                        
            else:
                for i in range(len(bwa_reads_info[key])):
                    bwa_entry = bwa_reads_info[key][i]
                    custom_entry = custom_reads_info[key][i]
                    
                    bwa_start = max(0, bwa_entry["start_ref"] - bwa_entry["length"])
                    bwa_end = min(fasta_seq_len, bwa_entry["start_ref"] + bwa_entry["length"])
                    if bwa_start <= custom_entry["start_ref"] and custom_entry["start_ref"] <= bwa_end:
                        corr_map_cnt += 1

    print("Comparison of position alignment:\n")

    print("Number of entries in bwa: " + str(n_bwa))
    print("Number of entries in custom: " + str(len(custom_reads_info)))
    
    print("% of reads from bwa existing in custom: " + str(float(custom_cnt) / float(n_bwa)))
    print("% of reads with correct mapping: " + str(corr_map_cnt / n_bwa))
    print("Number of reads with wrong number of appearence: " + str(len_diff_cnt))

    
# Program
is_test = 1
if is_test:
    compare_results("../data/MT.fa", "../data/bwa_res_test.sam", "../data/test.fq.samalike")
else:
    compare_results("../data/MT.fa", "../data/bwa_res.sam", "../data/art.fq.samalike")
print("done")

Comparison of position alignment:

Number of entries in bwa: 2488
Number of entries in custom: 2488
% of reads from bwa existing in custom: 1.0
% of reads with correct mapping: 1.0
Number of reads with wrong number of appearence: 0
done
